### **To Implement the application, run each cell step by step.**

In [ ]:
# Installing the Necessary Libraries (Ngork, Streamlit, and gdown)
!pip install -q pyngrok
!pip install -q streamlit
!pip install -q streamlit_ace
!pip install --upgrade --no-cache-dir gdown

In [ ]:
# Downloading the Feature Extractor
!gdown --id 1ExslKhLc8XV8T0S8lA2wBvHxrLs405vx

In [ ]:
# Downloading the Logistic Regression (LR) Classifier
!gdown --id 1-FkXZzLrOFRY2qbcI9BIouiap1264-ZT

In [ ]:
# Downloading Support Vector Machine-Radial Basis Function (SVM-RBF) Classifier
!gdown --id 1-A-4N0APxVN1YZKlVZz2GhUkvywYLMqC

In [ ]:
%%writefile app.py
# Importing the Necessary Libraries for Loading the Models 
import streamlit as st
import tensorflow as tf
import pickle
@st.cache(allow_output_mutation=True)

# The Function of Loading the Hybrid CNN Model for Feature Extraction
def load_feature_extractor():
  network=tf.keras.models.load_model('/content/ChestVGG_SGD.h5')
  model = tf.keras.models.Model(inputs=network.input, outputs=network.layers[41].output)
  return model

# The Function of Loading Classifiers
def load_svm():
  with open('/content/svc_rbf.pkl', 'rb') as file:  
    svc = pickle.load(file)
  return svc
def load_lr():
  with open('/content/lr.pkl', 'rb') as file:  
    lr = pickle.load(file)
  return lr

# Loading the Model
with st.spinner("The model is being loaded..."):
  feature_extractor=load_feature_extractor()
  svm=load_svm()
  lr=load_lr()

# The Title and User Guide
st.markdown("<h1 style='text-align: center; color: black;'>Pneumonia Detection in Chest X-Ray (CXR) Images</h1>", unsafe_allow_html=True)
st.write("This web application detects pneumonia cases with an accuracy of 98.55%.")
file = st.file_uploader("Please Upload a CXR Image (JPG, PNG, JPEG, or JFIF Format)", type=["jpg", "png", "jpeg", "jfif"])

# Importing the Necessary Libraries for Loading the Input Image and Making Predicitons
import cv2
from PIL import Image, ImageOps
import numpy as np
st.set_option('deprecation.showfileUploaderEncoding', False)

# The Function of Importing the Image and Predicting Its Corresponding Class 
def import_and_predict(image_data, feature_extractor, svm, lr):

        # Preparing the Image
        size = (224,224)    
        image = ImageOps.fit(image_data, size, Image.ANTIALIAS)
        image = np.asarray(image)
        img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        img=img/255.      
        img_reshape = img[np.newaxis,...]

        # Extracting Features of the Image for Predicting Using KNN Classifier
        features=feature_extractor.predict(img_reshape)

        # Making the Prediction Using KNN Classifier
        classifiers=[svm, lr]

        preds = [classifier.predict_proba(features) for classifier in classifiers]
        preds=np.array(preds)
        ideal_weights = [0.4, 0.1] # These optimal values were obtained after experimenting different values 

        #Use tensordot to sum the products of all elements over specified axes.
        ideal_weighted_preds = np.tensordot(preds, ideal_weights, axes=((0),(0)))
        ideal_weighted_ensemble_prediction = np.argmax(ideal_weighted_preds, axis=1)

        return ideal_weighted_ensemble_prediction

# Making the Prediciton
if file is None:
    st.text("Please upload an image file")
else:
  image = Image.open(file)
  st.image(image, use_column_width=True)
  prediction=import_and_predict(image, feature_extractor, svm, lr)
  result=""
  if prediction==0:
    st.success('Normal'.format(result))
  if prediction==1:
    st.success('Pneumonia'.format(result))

In [ ]:
# Downloading Ngrok Necessary Components 
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip

In [ ]:
# Installing Ngrok Necessary Components
!unzip ngrok-stable-linux-amd64.zip

### Create an Ngrok account using the following link: https://dashboard.ngrok.com/signup. 

### To get your Authtoken, go to the 'Your Authtoken' section in your Ngrok account and click on the 'Copy' button.

In [ ]:
# Use Your Authtoken ---> # Example: !./ngrok authtoken 29UX...X1
!./ngrok authtoken 261pNX6MxlKhXhxVxLRTPn2fZeU_Krs66Vu2njrMmwxQVLXx

In [ ]:
# Running the Application
!streamlit run app.py &>/dev/null&

### By Running the next cell, a public link is generated which can be used by anyone. Now, open the link with mobile phone browser, take or upload a Chest X-Ray (CXR) image, and get the output. 


In [ ]:
from pyngrok import ngrok
public_url = ngrok.connect('8501')
public_url